In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import os
import yaml
import pickle
from ipyfilechooser import FileChooser

from cryo_challenge.data import validate_input_config_disttodist, validate_config_dtd_optimal_q_kl
from cryo_challenge.data import DistributionToDistributionResultsValidator

After you compute the map to map distances between ground truth and your submission, you can quantify the similarity between the distributions, taking into account the population weights.

# 1. Create a config file
The config file (e.g. `config_files/config_distribution_to_distribution.yaml`) has information about how compare the distributions, including how to pool nearby ground truth states hypermarameters for optimizing the submitted population weights.

Here is an example of the config file

In [ ]:
# Select path to Distribution to Distribution config file
# An example of this file is available in the path ../config_files/config_distribution_to_distribution.yaml
config_d2d_path = FileChooser(os.path.expanduser("~"))
config_d2d_path.filter_pattern = '*.yaml'
display(config_d2d_path)

In [17]:
with open(config_d2d_path.value, "r") as file:
    config = yaml.safe_load(file)

config

{'input_fname': 'results/map_to_map_distance_matrix_submission_0.pkl',
 'metrics': ['l2', 'corr', 'bioem', 'fsc'],
 'gt_metadata_fname': 'data/metadata.csv',
 'n_replicates': 30,
 'n_pool_microstate': 5,
 'replicate_fraction': 0.9,
 'cvxpy_solver': 'ECOS',
 'optimal_q_kl': {'n_iter': 100000, 'break_atol': 0.0001},
 'output_fname': 'results/distribution_to_distribution_submission_0.pkl'}

These docstrings explain what the config file entries are

In [23]:
validate_input_config_disttodist?

Signature: validate_input_config_disttodist(config: dict) -> None
Docstring:
Validate the config dictionary.

input_fname: str, is the path to the map to map distance matrix (output from map2map_pipeline).
metrics: list, is a list of metrics to compute.
gt_metadata_fname: str, is the path to the ground truth metadata (.csv) file.
n_replicates: int, is the number of replicates to compute.
n_pool_microstate: int, is the number of microstates to pool (low values less than 3-5 can cause problems for optimization convergence in CVXPY numerical solvers).
replicate_fraction: float, is the fraction of the data to use for replicates.
cvxpy_solver: str, is the solver to use for CVXPY optimization.
optimal_q_kl: dict, is the optimal_q_kl part of the config.
File:      /mnt/ceph/users/gwoollard/repos/Cryo-EM-Heterogeneity-Challenge-1/src/cryo_challenge/data/_validation/config_validators.py
Type:      function

In [24]:
validate_config_dtd_optimal_q_kl?

Signature: validate_config_dtd_optimal_q_kl(config_optimal_q_kl: dict) -> None
Docstring:
Validate the optimal_q_kl part of the config dictionary for the DistributionToDistribution config.

n_iter: int, is the number of iterations for the optimization.
break_atol: float, is the absolute tolerance for the optimization.
File:      /mnt/ceph/users/gwoollard/repos/Cryo-EM-Heterogeneity-Challenge-1/src/cryo_challenge/data/_validation/config_validators.py
Type:      function

The whole config is validated in `cryo_challenge.data._validation.config_validators.validate_input_config_disttodist`.

In [25]:
validate_input_config_disttodist(config)

# 2. Run `run_distribution2distribution_pipeline`

`cryo_challenge run_distribution2distribution_pipeline --config config_files/config_distribution_to_distribution.yaml`

The output (of step #1) is validated with `cryo_challenge.data._validation.output_validators.DistributionToDistributionResultsValidator` (automatically, this is just explained below to serve as documentation)

In [26]:
with open(os.path.join('../',config["output_fname"]), "rb") as f:
    results_dict = pickle.load(f)


In [27]:
_ = DistributionToDistributionResultsValidator.from_dict(results_dict)

The fields of the output are explained below:

In [28]:
DistributionToDistributionResultsValidator?

Init signature:
DistributionToDistributionResultsValidator(
    config: dict,
    user_submitted_populations: torch.Tensor,
    id: str,
    fsc: Union[dict, NoneType] = None,
    bioem: Union[dict, NoneType] = None,
    l2: Union[dict, NoneType] = None,
    corr: Union[dict, NoneType] = None,
) -> None
Docstring:     
Validate the output dictionary of the distribution-to-distribution pipeline.

config: dict, input config dictionary.
user_submitted_populations: torch.Tensor, user submitted populations, which sum to 1.
id: str, submission id.
fsc: dict, FSC distance results.
bioem: dict, BioEM distance results.
l2: dict, L2 distance results.
corr: dict, correlation distance results.
File:           /mnt/ceph/users/gwoollard/repos/Cryo-EM-Heterogeneity-Challenge-1/src/cryo_challenge/data/_validation/output_validators.py
Type:           type
Subclasses:     

For each map to map distance, there is a dictionary of replicates

In [29]:
from cryo_challenge.data import MetricDistToDistValidator
MetricDistToDistValidator?

Init signature: MetricDistToDistValidator(replicates: dict) -> None
Docstring:     
Validate the output dictionary of one map to map metric in the the distribution-to-distribution pipeline.

replicates: dict, dictionary of replicates.
File:           /mnt/ceph/users/gwoollard/repos/Cryo-EM-Heterogeneity-Challenge-1/src/cryo_challenge/data/_validation/output_validators.py
Type:           type
Subclasses:     

For each replicate (subsampling the ground truth distribution), there is a separate dict of results containing: 

In [30]:
from cryo_challenge.data import ReplicateValidatorEMD, ReplicateValidatorKL

In [31]:
ReplicateValidatorEMD?

Init signature:
ReplicateValidatorEMD(
    q_opt: List[float],
    EMD_opt: float,
    transport_plan_opt: List[List[float]],
    flow_opt: Any,
    prob_opt: Any,
    runtime_opt: float,
    EMD_submitted: float,
    transport_plan_submitted: List[List[float]],
) -> None
Docstring:     
Validate the output dictionary of one EMD in the the distribution-to-distribution pipeline.

q_opt: List[float], optimal user submitted distribution, which sums to 1.
EMD_opt: float, EMD between the ground truth distribution (p) and the (optimized) user submitted distribution (q_opt). 
    The transport plan is a joint distribution, such that:
    summing over the rows gives the (optimized) user submitted distribution, and summing over the columns gives the ground truth distribution.
transport_plan_opt: List[List[float]], transport plan between the ground truth distribution (p, rows) and the (optimized) user submitted distribution (q_opt, columns).
flow_opt: cvx output, optimization output (problem is 

In [32]:
ReplicateValidatorKL?

Init signature:
ReplicateValidatorKL(
    q_opt: List[float],
    klpq_opt: float,
    klqp_opt: float,
    A: List[List[float]],
    iter_stop: int,
    eps_stop: float,
    klpq_submitted: float,
    klqp_submitted: float,
) -> None
Docstring:     
Validate the output dictionary of one KL divergence in the the distribution-to-distribution pipeline.

q_opt: List[float], optimal user submitted distribution, which sums to 1.
klpq_opt: float, KL divergence between the ground truth distribution (p) and the (optimized) user submitted distribution (q_opt).
klqp_opt: float, KL divergence between the (optimized) user submitted distribution and the ground truth distribution (p).
A: List[List[float]], assignment matrix.
iter_stop: int, number of iterations until convergence.
eps_stop: float, stopping criterion.
klpq_submitted: float, KL divergence between the ground truth distribution (p) and the user submitted distribution (q).
klqp_submitted: float, KL divergence between the user submitted di